# Sistema de Recomendação
Nesse notebook foi desenvolvido um sistema de recomendação baseado em similaridade. Ele recebe um título de qualquer produto, classifica-o e recomenda 10 produtos similares. O foco na parte de classificação será classificar produtos que não estão no dataset, pois caso contrário seria possível utilizar o classificador já desenvolvido no outro notebook. A avaliação desse sistema será feita de forma qualitativa.

O dataset utilizado pode ser encontrado nesse [link](https://elo7-datasets.s3.amazonaws.com/data_scientist_position/elo7_recruitment_dataset.csv), e ele foi baixado na data 28/03/2022. 

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from utils.estimators import FilterColumns
from utils.utils import tokenize, save_model

RANDOM_SEED = 0

In [2]:
df = pd.read_csv('../data/elo7_recruitment_dataset.csv')

In [3]:
df.head()

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração


Como nesse contexto apenas temos o 'title' do produto não cadastrado na base como input, foram utilizadas abordagens de similaride de texto utilizando o cálculo dos tf-idf e o cosseno entre vetores.

In [4]:
vars = ['product_id', 'title', 'view_counts', 'order_counts', 'query']
target = ['category']

In [5]:
X, y = df[vars], df[target]

In [6]:
tests = ['abajur', 'urso de pelucia', 'kit barba', 'anel', 'envelope']

def calc_tests(tests, model):
    ''' Calculate results for input tests
    Params:
    tests(list) - List of titles
    model(Estimator) - Model fit to test
    '''
    
    for idx, test in enumerate(tests):
        test_df = pd.DataFrame({'title': [test]})
        result = model.predict(test_df)
        
        print(f'Teste {idx}: {test}')
        
        for key, value in result.items():
                print(value)
        print()

## Abordagem 1

Na primeira abordagem para realizar a classificação do produto, foi utilizada uma estratégia parecida com o algoritmo KNN. Primeiro foi calculado os tf-idfs de todos os títulos de produto da base, depois com esse encoder treinado foi calculado o tf-idf para novo título. Após isso foi realizada o produto escalar entre os dois resultados para computar a similaridade entre o novo título e todos os títulos existentes. Então utiliza-se k produtos mais próximos do input, onde k nesse caso é 11 escolhido arbitrariamente apenas tomando cuidado do número ser ímpar para não ter empate, e a categoria majoritária entre esse grupo será a atribuída para o novo título.

Para o sistema de recomendação, foram recomendados os produtos mais clicados e mais vendidos da categoria do título do input.

In [7]:
class RecommendSystem_1(BaseEstimator):
    ''' 
    '''    
    def fit(self, X, y=None):
        
        self.products = X.join(y).copy()
        self.products.reset_index(drop=True, inplace=True)
        
        self.products = self.products.groupby(['product_id', 'title', 'category'])[
            ['view_counts', 'order_counts']].max().reset_index()
        
        self.pipe_tfidf = Pipeline([
            ('filter', FilterColumns('title')),
            ('vect', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf', TfidfTransformer())     
        ])
        
        self.tfidf_matrix = self.pipe_tfidf.fit_transform(self.products, y)
        
        return self
    
    def predict(self, X):
        # X is a series of titles
        title = X.copy()
        tfidf_string = self.pipe_tfidf.transform(title)
        tfidf_string = tfidf_string.todense()
        
        dense_tfidf_matrix = self.tfidf_matrix.todense()
        
        cos_similarity = np.dot(dense_tfidf_matrix, tfidf_string.T)
        
        cos_similarity = pd.DataFrame(cos_similarity,
                                      columns=['similarity'])
        
        self.sim_matrix = self.products.join(cos_similarity)
        
        # Predict category
        k = 11
        similar_titles_matrix = self.sim_matrix.sort_values(by='similarity', ascending=False)
        category = similar_titles_matrix.head(k).category.mode()[0]
        
        # Recommend top 10
        top_10 = self.sim_matrix.query(f"category=='{category}' & similarity!=1")
        top_10 = top_10.sort_values(['view_counts', 'order_counts'],
                           ascending=[False, False])
        
        top_10 = top_10.head(10)[['product_id', 'title']]
        
        top_10 = top_10.product_id.astype('str') + ','+ top_10.title
        
        result = {'category': category}
        
        for idx, product in enumerate(top_10):
            result['product_' + str(idx)] = product
                  
        return result

In [8]:
model_1 = RecommendSystem_1()

- Fit

In [9]:
model_1.fit(X, y)

RecommendSystem_1()

In [10]:
calc_tests(tests, model_1)

Teste 0: abajur
Decoração
15722213,Tapete simples em croche
495241,Poster A4 Salmo 91
2867175,Mesa Espelhada - Mesa de Centro - Promoção
2623238,Espelhos Decorativos em Acrílico 7 Peças | SQ07P
13917701,Presente dia dos Namorados Azulejo com Foto
12564194,Espelho Decorativo Acrílico Folhas
13818512,Mesa Cabeceira Criado Mudo em S
6832055,Jogo de banheiro Crochê
12584017,Pelego Fake Branco
611144,Presente aniversário de namoro casamento dia dos namorados

Teste 1: urso de pelucia
Bebê
6977307,Bolsa Maternidade Rose kit 4 peças mala bebe
9836291,kit bolsa mala bebe maternidade personalizada
9730357,Saída Maternidade Menina Anny 5 peças Salmão
13672802,kits higiene mdf
9739668,Kit Higiene Bebê Passa Fita 8 Pçs Mdf Cru Desmontado +Brinde
4755599,bolsa mala bebe de maternidade personalizada azul
12453653,Lembrancinha Chá De Bebe (par)
10208924,Nome Em MDF
16278919,ninho redutor de berço nome do bebe bordado
15127043,Caderneta de Vacinação 2019

Teste 2: kit barba
Lembrancinhas
8271183,Mini 

O classificador está funcionando bem nos exemplos testados, apesar que essa abordagem é bem custosa em quesito de processamento. Para a recomendação, recomendar os mais populares da mesma categoria está demonstrando muito aleatório.

## Abordagem 2

Na abordagem 2 foi mantido a mesma estratégia para o classificador, porém para o sistema de recomendação foram recomendados os 10 produtos com o título mais próximo do título do input utilizando novamento o td-idf e o cosseno.

In [11]:
class RecommendSystem_2(BaseEstimator):
    
    def fit(self, X, y=None):

        # Store products info
        self.products = X.join(y).copy()
        self.products.reset_index(drop=True, inplace=True)

        product_cols = ['product_id', 'title', 'category']
        self.products = self.products.groupby(product_cols)[
            ['view_counts', 'order_counts']].max().reset_index()
        
        self.pipe_tfidf = Pipeline([
            ('filter', FilterColumns('title')),
            ('vect', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf', TfidfTransformer())     
        ])

        # Fit tf-idf title transformer
        self.tfidf_matrix = self.pipe_tfidf.fit_transform(self.products, y)
        
        return self
    
    def predict(self, X):
        title = X.copy()
        
        # Calculate tf-idf for title input
        tfidf_string = self.pipe_tfidf.transform(title)
        tfidf_string = tfidf_string.todense()

        # Unpack title tf-idf matrix
        dense_tfidf_matrix = self.tfidf_matrix.todense()

        # Calculate cossine of title input vector and titles matrix
        cos_similarity = np.dot(dense_tfidf_matrix, tfidf_string.T)
        cos_similarity = pd.DataFrame(cos_similarity,
                                      columns=['similarity'])
        
        self.sim_matrix = self.products.join(cos_similarity)
        
        # Predict category
        k = 11
        similar_titles_matrix = self.sim_matrix.sort_values(by='similarity',
                                                            ascending=False)
        
        # Get majoritary class in selected group
        category = similar_titles_matrix.head(k).category.mode()[0]
        
        # Recommend top 10
        top_10 = similar_titles_matrix.query("similarity!=1")
        top_10 = top_10.head(10)[['product_id', 'title']]

        # Pack results in a dict
        top_10 = top_10.product_id.astype('str') + ','+ top_10.title

        result = {'category': category}
        for idx, product in enumerate(top_10):
            result['product_' + str(idx)] = product
                  
        return result

In [12]:
model_2 = RecommendSystem_2()

- Fit

In [13]:
model_2.fit(X, y)

RecommendSystem_2()

In [14]:
calc_tests(tests, model_2)

Teste 0: abajur
Decoração
8907076,Abajur de Laço Provençal - Abajur Menina- Abajur Mesa
1014210,Abajur MDF
8145510,Abajur Infantil
7058985,Abajur Infantil
1147680,Abajur infantil
2056028,Abajur Encanto
7494025,Luminária Abajur Gibi - Liga da Justiça
5229134,Abajur Corujinha
7544088,Abajur Twist
15015021,Abajur elefantinha

Teste 1: urso de pelucia
Bebê
14916286,Ursinhos de pelucia
3921315,Ursinhos de pelucia
4335481,Urso de Pelúcia Apaixonado 25cm
6443874,URSO DE PELUCIA
5391249,Ursinho de pelúcia
2024400,urso de pelucia
7340520,LEMBRANCINHA URSINHOS PELÚCIA-3 Cm
7454682,Ursinho de Pelúcia 9 cm personalizado
12621205,Mini Ursinho de pelúcia
563773,MINI URSINHO PELÚCIA CHAVEIRINHO-10 CMTS

Teste 2: kit barba
Lembrancinhas
9433066,Kit barba
6267864,Kit de Barbear
14050247,Kit barba
14128221,KIT BARBA
11768258,Kit de Barba
8320894,Kit Barba
10626864,Lembrancinha Kit de Barbear
10158126,Barbante Trento
7020996,Barbante trento
6020100,Shampoo de barba

Teste 3: anel
Bijuterias e Jóias
15311

Nessa abordagem a recomendação faz muito sentido com o produto do input, mas ela não possuí variedade e os produtos recomendados são muito parecidos.

## Abordagem 3

Manter novamente o classificador, porém para a recomendação usar o tf-idf do título de input para achar o produto mais parecido com ele. Com base na query que clicou naquele produto, achar as queries mais parecidas com aquela query, usando um outro vetor tf-idf, e recomendar os produtos clicados por aquelas queries. Nessa abordagem então são recomendados os produtos clicados com com consultas similares ao produto de input, essa estratégia visa recomendar produtos similares mas com uma variabilidade mais adequada.

In [15]:
class RecommendSystem_3(BaseEstimator):
    ''' 
    '''    
    def fit(self, X, y=None):
        
        # Store queries info
        self.queries = X[['product_id', 'title', 'query']].copy()
        self.queries.reset_index(drop=True, inplace=True)
        
        # Store products info
        self.products = X.join(y).copy()
        self.products.reset_index(drop=True, inplace=True)
        
        product_cols = ['product_id', 'title', 'category']
        self.products = self.products.groupby(product_cols).count()
        self.products.reset_index(inplace=True)
        
        # Fit tf-idf title transformer
        self.title_tfidf = Pipeline([
            ('filter', FilterColumns('title')),
            ('vect', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf', TfidfTransformer())     
        ])
        
        # Store tf-idf product titles matrix
        self.title_tfidf_matrix = self.title_tfidf.fit_transform(self.products, y)
        
        # Fit tf-idf query transformer
        self.query_tfidf = Pipeline([
            ('filter', FilterColumns('query')),
            ('vect', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf', TfidfTransformer())     
        ])
        
        # Store tf-idf queries matrix
        self.query_tfidf_matrix = self.query_tfidf.fit_transform(self.queries, y)
        
        return self

    def predict(self, X):
        title = X.copy()
        
        # Calculate tf-idf for title input
        tfidf_string = self.title_tfidf.transform(title)
        tfidf_string = tfidf_string.todense()
        
        # Unpack title tf-idf matrix
        title_tfidf_matrix = self.title_tfidf_matrix.todense()
        
        # Calculate cossine of title input vector and titles matrix
        title_similarity = np.dot(title_tfidf_matrix, tfidf_string.T)
        
        title_similarity = pd.DataFrame(title_similarity,
                                      columns=['similarity'])
        
        self.sim_matrix = self.products.join(title_similarity)
        
        # Predict category
        k = 11
        similar_title_matrix = self.sim_matrix.sort_values(by='similarity', ascending=False)
        
        category = similar_title_matrix.head(k).category.mode()[0]
        
        # Recommend top 10
        
        # Get most similar product with title
        product_id = similar_title_matrix.product_id.head(1).values[0]
        
        # Get a random query that was used to find the product
        self.similar_query = self.queries.query(f'product_id=={product_id}').head(1)
        
        # Calculate tf-idf vector of this query
        similar_query = self.query_tfidf.transform(self.similar_query[['query']])
        similar_query = similar_query.todense()
        
        # Unpack query tf-idf matrix
        query_tfidf_matrix = self.query_tfidf_matrix.todense()
        
        # Calculate cossine of title input vector and query matrix
        query_similarity = np.dot(query_tfidf_matrix, similar_query.T)
        query_similarity = pd.DataFrame(query_similarity, columns=['similarity'])
        
        query_similarity = self.queries.join(query_similarity)
        query_similarity.sort_values(by=['similarity'], ascending=False, inplace=True)
        
        # Get top 10 similar queries excluding the exact one
        top_10 = query_similarity[round(query_similarity.similarity, 2) != 1][['product_id', 'title']].head(10)
        
        # Pack results in a dict
        top_10 = top_10.product_id.astype('str') + ','+ top_10.title
        
        result = {'category': category}
        
        for idx, product in enumerate(top_10):
            result['product_' + str(idx)] = product
            
        return result

In [16]:
model_3 = RecommendSystem_3()

- Fit

In [17]:
model_3.fit(X, y)

RecommendSystem_3()

In [18]:
calc_tests(tests, model_3)

Teste 0: abajur
Decoração
8907076,Abajur de Laço Provençal - Abajur Menina- Abajur Mesa
7058985,Abajur Infantil
8145510,Abajur Infantil
1147680,Abajur infantil
9694189,Abajur Infantil Coroa / príncipe
2728385,Abajour mdf
16099604,Abajur MDF Urso e Ramo Luxo
6316182,Abajur MDF Safari Feltro mod. 1
11836315,Abajour mdf
7697949,Abajour mdf

Teste 1: urso de pelucia
Bebê
12621205,Mini Ursinho de pelúcia
7340520,LEMBRANCINHA URSINHOS PELÚCIA-3 Cm
13325068,Chaveiro mini ursinho pelúcia-6 cmts
2784076,MINI URSINHOS DE PELÚCIA C COROA E LACINHO
5497840,PELÚCIA CHAVEIRO-MINI URSINHO
7454682,Ursinho de Pelúcia 9 cm personalizado
563773,MINI URSINHO PELÚCIA CHAVEIRINHO-10 CMTS
3569024,Pelúcias de Safári.
9608343,coelhinhos de pelúcia
10411706,Ovelha de Pelucia

Teste 2: kit barba
Lembrancinhas
8245994,Kit 10 unidades barbante euroroma
16524741,Barbante EuroRoma pink fio 6
14774354,Jogo de banheiro em barbante
12070350,jogo de banheiro de barbante
11844405,tapete de crochê Elos em barbante
1160432

Nessa abordagem o sistema começa a divergir a intenção do produto, como 'barba' acaba trazendo coisas relacionadas a 'barbante'.
Mesmo a segunda opção tendo pouca variabilidade, ela ainda é mais acertiva. Portanto, o modelo final escolhido será o da abordagem 2.

In [19]:
# Save recommender model
final_model = {'name': 'recommend_system',
               'version':1.0,
               'model': model_2}

save_model(final_model, '../models/recommender.pkl')

## Conclusão

A estratégia para o classificador se mostrou acertiva para os casos testados, porém a recomendação precisa ser melhorada. É possível recomendar produtos bem parecidos com o input, mas seria interessante que eles variassem mais, trazessem novidades e tendências também.

Como trabalhos futuros de hipóteses que não foram testas nesse estudo:
- Utilizar a informação de concatenated_tags para criar subcategorias dos produtos, sendo assim seria possível recomendar dentro de uma categoria produtos de subcategorias diferentes.
- Testar a criação de um espaço vetorial de texto único entre 'query' e 'title', supondo que os termos de 'query' são tentativas de digitar o nome do produto. Isso poderia melhorar as abordagens de cálculo de similaridade.
- Se der resultados a segunda hipótese, testar 'concatenated_tags' em conjunto com o 'query' e 'title'.